<a href="https://colab.research.google.com/github/Vijay-K-2003/mlp/blob/main/LightningModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Lightning

In [1]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 43.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

# Import Packages

In [2]:
import os

In [3]:
import torch
from torch import nn

In [4]:
from torch.nn import functional as F

In [5]:
from torch.utils.data import TensorDataset, DataLoader, random_split

In [6]:
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

In [7]:
import lightning as L
L.seed_everything(13, workers=True)
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

INFO: Seed set to 13
INFO:lightning.fabric.utilities.seed:Seed set to 13


In [8]:
model_path = 'pytorch-lightning/'

In [9]:
!rm -rf pytorch-lightning
!ls -al

total 16
drwxr-xr-x 1 root root 4096 Apr 11 13:21 .
drwxr-xr-x 1 root root 4096 Apr 13 06:29 ..
drwxr-xr-x 4 root root 4096 Apr 11 13:21 .config
drwxr-xr-x 1 root root 4096 Apr 11 13:21 sample_data


# Model Creation

In [10]:
import torchmetrics

In [23]:
class LightningMNISTClassifier(L.LightningModule):
  def __init__(self, lr):
    super(LightningMNISTClassifier, self).__init__()

    self.layer1 = nn.Linear(28 * 28, 128)
    self.layer2 = nn.Linear(128, 256)

    self.lr = lr

    self.classifier = nn.Linear(256, 10)
    self.loss = nn.CrossEntropyLoss()

    self.train_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
    self.train_acc.reset()

    self.val_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
    self.val_acc.reset()

    self.test_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
    self.test_acc.reset()

    self.train_loss = []
    self.val_loss = []
    self.test_loss = []

  def forward(self, image):
    batch_size, channels, width, height = image.size()

    image = image.view(batch_size, -1)

    intermediate = self.layer1(image)
    intermediate = torch.relu(intermediate)

    intermediate = self.layer2(intermediate)
    intermediate = torch.relu(intermediate)

    return(intermediate)

  def training_step(self, train_batch, batch_idx):
    images, labels = train_batch

    features = self.forward(images)
    logits = self.classifier(features)

    train_loss = self.loss(logits, labels)
    self.train_loss.append(train_loss)

    self.train_acc(logits, labels)

    self.log("train_loss", train_loss, on_step=True)

    return train_loss

  def on_train_epoch_end(self):
    train_loss_mean = torch.mean(torch.tensor(self.train_loss))

    train_acc_mean = self.train_acc.compute()

    self.log("train_loss", train_loss_mean, on_epoch=True, sync_dist=True)
    self.log("train_acc", train_acc_mean, on_epoch=True, sync_dist=True)

    self.print("train_loss", train_loss_mean.item(), "train_acc", train_acc_mean.item())

    self.train_loss = []
    self.train_acc.reset()

  def validation_step(self, val_batch, batch_idx):
    images, labels = val_batch

    features = self.forward(images)
    logits = self.classifier(features)

    val_loss = self.loss(logits, labels)
    self.val_loss.append(val_loss)

    self.val_acc(logits, labels)

    self.log("val_loss", val_loss, on_step=True)

    return val_loss

  def on_validation_epoch_end(self):
    val_loss_mean = torch.mean(torch.tensor(self.val_loss))

    val_acc_mean = self.val_acc.compute()

    self.log("val_loss", val_loss_mean, on_epoch=True, sync_dist=True)
    self.log("val_acc", val_acc_mean, on_epoch=True, sync_dist=True)

    self.print("val_loss", val_loss_mean.item(), "val_acc", val_acc_mean.item())

    self.val_loss = []
    self.val_acc.reset()

  def test_step(self, test_batch, batch_idx):
    images, labels = test_batch

    features = self.forward(images)
    logits = self.classifier(features)

    test_loss = self.loss(logits, labels)
    self.test_loss.append(test_loss)

    self.test_acc(logits, labels)

    self.log("test_loss", test_loss, on_step=True)

    return test_loss

  def on_test_epoch_end(self):
    test_loss_mean = torch.mean(torch.tensor(self.test_loss))
    test_acc_mean = self.test_acc.compute()

    self.log("test_loss", test_loss_mean, on_epoch=True, sync_dist=True)
    self.log("test_acc", test_acc_mean, on_epoch=True, sync_dist=True)

    self.print("test_loss", test_loss_mean.item(), "test_acc", test_acc_mean.item())

    self.test_loss = []
    self.test_acc.reset()

  def lr_scheduler_step(self, scheduler, metric):
    print('lr_scheduler_step', metric.item())
    scheduler.step(metric)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           mode='min',
                                                           factor=0.1,
                                                           patience=2,
                                                           threshold=0.01,
                                                           threshold_mode='rel',
                                                           min_lr=1e-7,
                                                           verbose=True)
    lr_scheduler = {
        'scheduler': scheduler,
        'monitor': 'val_loss',
        'interval': 'epoch'
        }
    return( [optimizer], [lr_scheduler] )

# Define Custom Callback

In [24]:
class PrintCallback(L.pytorch.callbacks.Callback):

    def on_init_start(self, trainer):
        print('Starting to initialize trainer!')

    def on_init_end(self, trainer):
        print('Trainer is initialized !!!')

    def on_train_end(self, trainer, pl_module):
        print('Training Ends here !!!')


# MNIST Data Preparation

In [25]:

def prepare_data():
  # transforms for images
  transform=transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

  # prepare transforms standard to MNIST
  mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
  mnist_train = [mnist_train[i] for i in range(2200)]

  mnist_train, mnist_val = random_split(mnist_train, [2000, 200])

  mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
  mnist_test = [mnist_test[i] for i in range(3000,4000)]

  return mnist_train, mnist_val, mnist_test

In [26]:
train, val, test = prepare_data()

# Data Loader For MNIST

In [27]:
train_loader, val_loader, test_loader = DataLoader(train, batch_size=64), DataLoader(val, batch_size=64), DataLoader(test, batch_size=64)

# Train the Model

In [28]:
model = LightningMNISTClassifier(lr=1e-3)

In [29]:
lr_logger = LearningRateMonitor()

In [30]:
early_stopping = EarlyStopping('val_loss', mode='min', patience=10)

In [31]:
checkpoint_callback = ModelCheckpoint(
    filename=model_path+'mnist_{epoch}-{val_loss:.5f}',
    monitor='val_loss',
    mode='min',
    save_top_k=3
    )

In [32]:
trainer = L.Trainer(
    max_epochs=150,
    callbacks=[lr_logger, checkpoint_callback],
    default_root_dir=model_path
    )

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [33]:
trainer.fit(model, train_loader, val_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
INFO: 
  | Name       | Type               | Params
--------------------------------------------------
0 | layer1     | Linear             | 100 K 
1 | layer2     | Linear             | 33.0 K
2 | classifier | Linear             | 2.6 K 
3 | loss       | CrossEntropyLoss   | 0     
4 | train_acc  | MulticlassAccuracy | 0     
5 | val_acc    | MulticlassAccuracy | 0     
6 | test_acc   | MulticlassAccuracy | 0     
--------------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)
INFO:lightnin

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

val_loss tensor(2.2975) val_acc tensor(0.0703, device='cuda:0')


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (32) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

val_loss tensor(0.1870) val_acc tensor(0.9250, device='cuda:0')
train_loss tensor(0.0213) train_acc tensor(0.9990, device='cuda:0')
lr_scheduler_step 0.18696928024291992


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=150` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=150` reached.


val_loss tensor(0.1870) val_acc tensor(0.9250, device='cuda:0')
train_loss tensor(0.0213) train_acc tensor(0.9990, device='cuda:0')
lr_scheduler_step 0.18696850538253784


In [34]:
print("State Dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

State Dict:
layer1.weight 	 torch.Size([128, 784])
layer1.bias 	 torch.Size([128])
layer2.weight 	 torch.Size([256, 128])
layer2.bias 	 torch.Size([256])
classifier.weight 	 torch.Size([10, 256])
classifier.bias 	 torch.Size([10])


# Tensorboard

In [36]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9071), started 0:00:20 ago. (Use '!kill 9071' to kill it.)

<IPython.core.display.Javascript object>

# Validate Model

In [37]:
trainer.validate(model, val_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

val_loss tensor(0.1870) val_acc tensor(0.9250, device='cuda:0')


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │     0.925000011920929     │
│         val_loss          │    0.18696850538253784    │
│      val_loss_epoch       │    0.23790548741817474    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss_epoch': 0.23790548741817474,
  'val_loss': 0.18696850538253784,
  'val_acc': 0.925000011920929}]

# Test Model

In [38]:
trainer.test(model, test_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

test_loss tensor(0.4803) test_acc tensor(0.8690, device='cuda:0')


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8690000176429749     │
│         test_loss         │    0.4803493022918701     │
│      test_loss_epoch      │    0.4859990179538727     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.4859990179538727,
  'test_loss': 0.4803493022918701,
  'test_acc': 0.8690000176429749}]